In [1]:
#!pip install xlsxwriter

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
df = pd.read_excel('CBS_CRM.xlsx', index_col=0, skiprows=1)

In [4]:
df.drop('Unnamed: 15', axis=1, inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1674 entries, 1.0 to nan
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATA 1º CONTATO   1672 non-null   object        
 1   NOME COMPLETO     1670 non-null   object        
 2   IDADE             1428 non-null   object        
 3   TELEFONE          1591 non-null   object        
 4   STATUS            1668 non-null   object        
 5   VIA               1668 non-null   object        
 6   ESTILOS           1664 non-null   object        
 7   DATA AGENDAMENTO  223 non-null    datetime64[ns]
 8   HORÁRIO           1301 non-null   object        
 9   PROFESSOR         1373 non-null   object        
 10  COMPARECEU        1650 non-null   object        
 11  REAGENDAMENTO     37 non-null     object        
 12  SITUAÇÃO          1640 non-null   object        
 13  OBSERVAÇÕES       838 non-null    object        
dtypes: datetime64[ns](1), object

In [6]:
print(df.isnull().sum())

DATA 1º CONTATO        2
NOME COMPLETO          4
IDADE                246
TELEFONE              83
STATUS                 6
VIA                    6
ESTILOS               10
DATA AGENDAMENTO    1451
HORÁRIO              373
PROFESSOR            301
COMPARECEU            24
REAGENDAMENTO       1637
SITUAÇÃO              34
OBSERVAÇÕES          836
dtype: int64


In [7]:
df['DATA 1º CONTATO'] = df['DATA 1º CONTATO'].astype(str) #converter em string para alterações

In [8]:
df.dropna(subset=['DATA 1º CONTATO'], inplace=True) #dropar nulos

In [9]:
df = df[df['DATA 1º CONTATO'].astype(str).str.replace(' |-', '', regex=True).ne('')] #corrigir valores que não estão no formato

In [10]:
df['DATA 1º CONTATO'] = df['DATA 1º CONTATO'].str.replace('223', '2023').str.replace('2012', '2021') #corrigindo inconsistencias

In [11]:
#df['DATA 1º CONTATO'].unique()

In [12]:
df = df[pd.to_datetime(df['DATA 1º CONTATO'], errors='coerce').notna()] # dropar nulos
df['DATA 1º CONTATO'] = pd.to_datetime(df['DATA 1º CONTATO'], errors='coerce').dt.date  #converter em formato date

<ipython-input-12-cb89492dbf02>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATA 1º CONTATO'] = pd.to_datetime(df['DATA 1º CONTATO'], errors='coerce').dt.date  #converter em formato date


In [13]:
df.dropna(subset=['NOME COMPLETO'], inplace=True)  # Excluir valores nulos
df['NOME COMPLETO'] = df['NOME COMPLETO'].str.replace(r'[^\w\s]', '', regex=True)  # Remover caracteres especiais
df['NOME COMPLETO'] = df['NOME COMPLETO'].str.strip()  # Remover espaços em branco

<ipython-input-13-975c559e56f9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['NOME COMPLETO'], inplace=True)  # Excluir valores nulos
<ipython-input-13-975c559e56f9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NOME COMPLETO'] = df['NOME COMPLETO'].str.replace(r'[^\w\s]', '', regex=True)  # Remover caracteres especiais
<ipython-input-13-975c559e56f9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [14]:
# Converter valores numéricos e '%-%' para float, e strings para NaN
df['IDADE'] = df['IDADE'].apply(lambda x: float(x) if isinstance(x, (int, float)) else
                                      (float(x.split('-')) if isinstance(x, str) and '-' in x else np.nan))

# Preencher valores NaN com a média da coluna
df['IDADE'].fillna(df['IDADE'].mean(), inplace=True)

<ipython-input-14-511440e22d5e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IDADE'] = df['IDADE'].apply(lambda x: float(x) if isinstance(x, (int, float)) else
<ipython-input-14-511440e22d5e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IDADE'].fillna(df['IDADE'].mean(), inplace=True)


In [15]:
# Crie a coluna 'FAIXA ETARIA'
df['FAIXA ETARIA'] = df['IDADE'].apply(lambda x:
    'Criança' if x < 10 else
    'Adolescente' if x <= 15 else
    'Adulto'
)

<ipython-input-15-72bb39f14d4c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['FAIXA ETARIA'] = df['IDADE'].apply(lambda x:


In [16]:
df.head()

,DATA 1º CONTATO,NOME COMPLETO,IDADE,TELEFONE,STATUS,VIA,ESTILOS,DATA AGENDAMENTO,HORÁRIO,PROFESSOR,COMPARECEU,REAGENDAMENTO,SITUAÇÃO,OBSERVAÇÕES,FAIXA ETARIA
QTDE,,,,,,,,,,,,,,,
1.0,2022-01-07,Graziele,22.000000,12 99614-0592,AULA EXPERIMENTAL,WHATSAPP,JAZZ FUNK,NaT,18h30,ALISSON,SIM,NaN,MATRICULADO,NaN,Adulto
2.0,2022-01-07,Camila,19.721145,11 94129-6714,INFORMAÇÕES,WHATSAPP,JAZZ,NaT,19H30,LIA,NÃO,NaN,SEMANA SOLIDÁRIA,NaN,Adulto
3.0,2021-01-05,MAIARA GOUVEA,16.000000,98841-5432,AULA EXPERIMENTAL,WHATS,K-POP,NaT,19H00,ALISSON,NÃO FICOU,NaN,NÃO RESPONDE,NaN,Adulto
4.0,2021-01-05,ENIALE RAMOS QUINTINO MÃE ELAINE,11.000000,99124-9509,AULA EXPERIMENTAL,WHATS,JAZZ,NaT,17H00,LIA,MATRICULADO,NaN,MATRICULADO,NaN,Adolescente
5.0,2022-01-07,Bia,19.721145,12 98129-7939,INFORMAÇÕES,WHATSAPP,MODALIDADES,NaT,-,-,NÃO,NaN,SEMANA SOLIDÁRIA,NaN,Adulto


In [17]:
# Ignore os caracteres especiais
df['TELEFONE'] = df['TELEFONE'].str.replace(r'[^\d]', '')

# Padronize os valores nulos com '0000-0000'
df['TELEFONE'].fillna('0000-0000', inplace=True)

In [18]:
df['STATUS'].unique()

array(['AULA EXPERIMENTAL', 'INFORMAÇÕES', 'AULA ESPERIMENTAL',
       'INFORMAÇÕES ', 'CAPTAÇÃO', 'COREOGRAFIA', 'AULA AVULSA',
       'INFORMACOES', 'INFORMAÇÃO', 'INFORMAÇOES', nan], dtype=object)

In [19]:
# Padronize 'INFORMAÇÕES'
df['STATUS'] = df['STATUS'].str.replace('INFORMAÇÕES ', 'INFORMAÇÕES')
df['STATUS'] = df['STATUS'].str.replace('INFORMACOES', 'INFORMAÇÕES')
df['STATUS'] = df['STATUS'].str.replace('INFORMAÇÃO', 'INFORMAÇÕES')
df['STATUS'] = df['STATUS'].str.replace('INFORMAÇOES', 'INFORMAÇÕES')
df['STATUS'] = df['STATUS'].str.replace('INFORMAÇÕES', 'CAPTAÇÃO')
# Substitua 'AULA AVULSA' por 'EXPERIMENTAL'
df['STATUS'] = df['STATUS'].str.replace('AULA AVULSA', 'EXPERIMENTAL')
df['STATUS'] = df['STATUS'].str.replace('AULA EXPERIMENTAL', 'EXPERIMENTAL')

# Padronize 'AULA ESPERIMENTAL'
df['STATUS'] = df['STATUS'].str.replace('AULA ESPERIMENTAL', 'EXPERIMENTAL')
df['STATUS'] = df['STATUS'].str.replace('AULA AULA EXPERIMENTAL', 'EXPERIMENTAL')
df['STATUS'] = df['STATUS'].str.replace('COREOGRAFIA', 'EXPERIMENTAL')


# excluir nulos
df = df.dropna(subset=['STATUS'])

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1665 entries, 1.0 to nan
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATA 1º CONTATO   1665 non-null   object        
 1   NOME COMPLETO     1665 non-null   object        
 2   IDADE             1665 non-null   float64       
 3   TELEFONE          1665 non-null   object        
 4   STATUS            1665 non-null   object        
 5   VIA               1665 non-null   object        
 6   ESTILOS           1661 non-null   object        
 7   DATA AGENDAMENTO  222 non-null    datetime64[ns]
 8   HORÁRIO           1297 non-null   object        
 9   PROFESSOR         1370 non-null   object        
 10  COMPARECEU        1647 non-null   object        
 11  REAGENDAMENTO     37 non-null     object        
 12  SITUAÇÃO          1636 non-null   object        
 13  OBSERVAÇÕES       838 non-null    object        
 14  FAIXA ETARIA      1665 non-n

In [21]:
df['VIA'].unique()

array(['WHATSAPP', 'WHATS', 'INSTAGRAM', 'TELEFONE', 'PESSOALMENTE',
       'PRESENCIAL', 'PRESENCIAL ', 'SITE', 'VALE AULA', 'FACEBOOK',
       'MESSENGER', 'MAE JÁ É ALUNA', 'ALUNA'], dtype=object)

In [22]:
#padrozniar
df['VIA'] = df['VIA'].str.replace('WHATSAPPAPP', 'WHATSAPP')
df['VIA'] = df['VIA'].str.replace('WHATSAPP', 'WHATSAPP')
df['VIA'] = df['VIA'].str.replace('TELEFONE', 'WHATSAPP')
df['VIA'] = df['VIA'].str.replace('PESSOALMENTE', 'PRESENCIAL')
df['VIA'] = df['VIA'].str.replace('VALE ', 'PRESENCIAL')
df['VIA'] = df['VIA'].str.replace('MAE JÁ É ALUNA', 'PRESENCIAL')
df['VIA'] = df['VIA'].str.replace('ALUNA', 'PRESENCIAL')
df['VIA'] = df['VIA'].str.replace('MESSENGER', 'FACEBOOK')
df['VIA'] = df['VIA'].str.replace('PRESENCIALAULA', 'PRESENCIAL')
df['VIA'] = df['VIA'].str.replace('PRESENCIAL ', 'PRESENCIAL')

In [23]:
# padronização
df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZ FUNK ', 'JAZZ FUNK')
df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZ FUNKINICIANTE', 'JAZZ FUNK')
df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZ FUNKINTERMEDIARIO', 'JAZZ FUNK')
df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZFUNK', 'JAZZ FUNK')


df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZ ', 'JAZZ')
df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZ INICIANTE', 'JAZZ')
df['ESTILOS'] = df['ESTILOS'].str.replace('Jazz', 'JAZZ')
df['ESTILOS'] = df['ESTILOS'].str.replace('Jazz iniciante', 'JAZZ')
df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZINICIANTE', 'JAZZ')
df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZ iniciante', 'JAZZ')
df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZiniciante', 'JAZZ')

df['ESTILOS'] = df['ESTILOS'].str.replace('AULA DE BREAK', 'DANÇAS URBANAS')
df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZE D URBANAS', 'DANÇAS URBANAS')


In [24]:
df['ESTILOS'] = df['ESTILOS'].str.replace(r'[^\w\s]+', '', regex=True).str.replace('\s+', ' ', regex=True).str.strip()

In [25]:
df['ESTILOS'] = df['ESTILOS'].str.replace(r'BALLET.*|KPOPBALLET.*|BALLEYT', 'BALLET CLÁSSICO', regex=True)
df['ESTILOS'] = df['ESTILOS'].str.replace(r'KPOP.*|K POP INTERMEDIARIO|K POP INICIANTE|kpop|K Pop|Kpop|K POP', 'K-POP', regex=True)
df['ESTILOS'] = df['ESTILOS'].str.replace(r'SAPATEADO.*|Sapateado', 'SAPATEADO', regex=True)
df['ESTILOS'] = df['ESTILOS'].str.replace(r'CONTEMPORANEO.*', 'CONTEMPORÂNEO', regex=True)

In [26]:
# Contar a frequência de cada valor em ESTILO
value_counts = df['ESTILOS'].value_counts()

In [27]:
# Selecionar os 3 valores mais frequentes
top_3_values = value_counts.nlargest(3).index.tolist()

In [28]:
top_3_values

['DANÇAS URBANAS', 'JAZZ', 'JAZZFUNK']

In [29]:
# Padronizar os valores que não são "JAZZFUNK", "JAZZ", "K-POP", "SAPATEADO", "DANÇAS URBANAS", "CONTEMPORÂNEO" e "BALLET CLÁSSICO" e distribuir igualitariamente entre estes os 3 valores que mais aparecem
df['ESTILOS'] = np.where(df['ESTILOS'].isin(['JAZZFUNK', 'JAZZ', 'K-POP', 'SAPATEADO', 'DANÇAS URBANAS', 'CONTEMPORÂNEO', 'BALLET CLÁSSICO']),
                        df['ESTILOS'],
                        np.random.choice(top_3_values))

In [30]:
df['DATA AGENDAMENTO'] = df['DATA AGENDAMENTO'].astype(str)

In [31]:
# Preencha os valores vazios com "Não agendou"
df['DATA AGENDAMENTO'].fillna('Não agendou', inplace=True)


# Altere o ano de 2032 para 2023
df['DATA AGENDAMENTO'] = df['DATA AGENDAMENTO'].apply(lambda x: '2023-' + x[5:] if x and x.startswith('2032') else x)

In [32]:
#df['DATA AGENDAMENTO'] = pd.to_datetime(df['DATA AGENDAMENTO'], errors='coerce').dt.date  #converter em formato date

In [33]:
# Renomear a coluna HORÁRIO para TURNO
df = df.rename(columns={'HORÁRIO': 'TURNO'})

In [34]:
# função que converte um horário em um turno do dia
def convert_horario(x):
    if isinstance(x, str):
        if '-' in x or x[:2].isalpha():
            return 'Unknown'
        else:
            hour = ''.join(filter(str.isdigit, x))
            if hour:
                hour = int(hour)
                if 6 <= hour <= 12:
                    return 'MANHA'
                elif 12 <= hour <= 18:
                    return 'TARDE'
                else:
                    return 'NOITE'
            else:
                return 'Unknown'
    else:
        return 'Unknown'

df['TURNO'] = df['TURNO'].apply(convert_horario)

In [35]:
# Substituição dos valores 'Unknown' pela moda
moda_turno = df['TURNO'].mode()[0]
df['TURNO'] = df['TURNO'].replace('Unknown', moda_turno)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1665 entries, 1.0 to nan
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   DATA 1º CONTATO   1665 non-null   object 
 1   NOME COMPLETO     1665 non-null   object 
 2   IDADE             1665 non-null   float64
 3   TELEFONE          1665 non-null   object 
 4   STATUS            1665 non-null   object 
 5   VIA               1665 non-null   object 
 6   ESTILOS           1665 non-null   object 
 7   DATA AGENDAMENTO  1665 non-null   object 
 8   TURNO             1665 non-null   object 
 9   PROFESSOR         1370 non-null   object 
 10  COMPARECEU        1647 non-null   object 
 11  REAGENDAMENTO     37 non-null     object 
 12  SITUAÇÃO          1636 non-null   object 
 13  OBSERVAÇÕES       838 non-null    object 
 14  FAIXA ETARIA      1665 non-null   object 
dtypes: float64(1), object(14)
memory usage: 208.1+ KB


In [37]:
df['PROFESSOR'].unique()

array(['ALISSON', 'LIA', '-', 'BIANCA', 'NICOLAS', 'FERNANDO', 'GIBELI',
       'PAOLA', nan, 'GABRIELA', 'LOUBACH', 'ALISSIN', 'ALISSON ', 'LIA ',
       'CAROL', 'ISABELLY', 'LARISSA', 'AMANDA', 'CICERO', 'ALINE',
       'ALLISON ', 'CÍCERO', 'NÃO', 'ALISSON/LARI', 'TODAS', 'NÃO ',
       'LARISSA / ALISSON', 'FELIPE'], dtype=object)

In [ ]:
df['PROFESSOR'] = df['PROFESSOR'].str.replace(r'ALISSIN.*|ALISSON .*|ALLISON ', 'ALISSON', regex=True)
df['ESTILOS'] = df['ESTILOS'].str.replace('LIA ', 'LIA')
df['PROFESSOR'] = df['PROFESSOR'].str.replace(r'BALLET.*|KPOPBALLET.*|BALLEYT', 'BALLET CLÁSSICO', regex=True)
df['PROFESSOR'] = df['PROFESSOR'].str.replace(r'BALLET.*|KPOPBALLET.*|BALLEYT', 'BALLET CLÁSSICO', regex=True)
df['PROFESSOR'] = df['PROFESSOR'].str.replace(r'BALLET.*|KPOPBALLET.*|BALLEYT', 'BALLET CLÁSSICO', regex=True)
df['PROFESSOR'] = df['PROFESSOR'].str.replace(r'BALLET.*|KPOPBALLET.*|BALLEYT', 'BALLET CLÁSSICO', regex=True)

In [38]:
with pd.ExcelWriter('CRM.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='CRM', index=False)
    #df_pc.to_excel(writer, sheet_name='Plano de contas', index=False)